In [1]:
# imports
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import timezone
from datetime import datetime as dt
import numpy as np
import sys
from scipy.stats import pearsonr
from scipy import stats

sys.path.append("../")
from src.utils import *

In [ ]:
# polars config
pl.Config.load_from_file("../polars_cfg.json")

# 1. Data loading

Choose whether to include participants that didn't complete all five tasks in the analyses with the True / False toggle.


In [19]:
# data file paths
clean_data_path = "../data/data.csv"

In [20]:
df_all = pl.read_csv(clean_data_path)

In [ ]:
df_all.head()

In [ ]:
sorted_df = df_all.sort(
    by=["exp_name", "joint_id_task", "task_number", "attempt_number", "action_id"]
)
aggregated_df = sorted_df.group_by(
    "joint_id_task", "attempt_number", maintain_order=True
).agg(pl.col("test_input_grid"))

In [ ]:
output_json_path = "../data/aggregated_test_input_grid.json"

# Write the aggregated DataFrame to a JSON file
aggregated_df.write_json(output_json_path, row_oriented=True)